# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "FTPMicrodadosNovoCaged"
extrator_bronze = AlinharETL(bucket,datamart)

2024-09-25 19:03:59,314 - INFO - Iniciando Sessão Spark.


# Leitura da tabela

In [3]:
df = extrator_bronze.criar_view_temporaria('bronze/NovoCaged/MicrodadosNovoCagedTipoMovimentacao','compliance_tipomovimentacao')

2024-09-25 19:04:04,581 - INFO - Aguarde... Criando View (bronze/NovoCaged/MicrodadosNovoCagedTipoMovimentacao)
2024-09-25 19:04:53,521 - INFO - View criada com sucesso


# Tratamentos na tabela

In [4]:
sql_query = """
    SELECT 
        CAST(`Código` AS INT) AS codigo, 
        `Descrição` AS descricao 
    FROM compliance_tipomovimentacao
"""

In [5]:
df = extrator_bronze.carregar_dados_delta(sql_query)

2024-09-25 19:04:53,536 - INFO - Aguarde... Executando Query (Delta)
2024-09-25 19:04:53,537 - INFO - 
    SELECT 
        CAST(`Código` AS INT) AS codigo, 
        `Descrição` AS descricao 
    FROM compliance_tipomovimentacao

2024-09-25 19:04:53,611 - INFO - Query (Delta) executada com sucesso


# Gravação no datalake

In [6]:
extrator_bronze.caminho_tabela_delta = 's3a://silver/NovoCaged/MicrodadosNovoCagedTipoMovimentacao'

In [7]:
extrator_bronze.salvar_delta(df, 'overwrite')

2024-09-25 19:04:53,625 - INFO - Aguarde... Persistindo dados (overwrite)
2024-09-25 19:05:03,370 - INFO - Dados persistidos com sucesso
2024-09-25 19:05:03,371 - INFO - s3a://silver/NovoCaged/MicrodadosNovoCagedTipoMovimentacao
2024-09-25 19:05:03,372 - INFO - Aguarde... Realizando optimize
2024-09-25 19:05:06,259 - INFO - Optimize executado com sucesso.
2024-09-25 19:05:06,260 - INFO - Aguarde... Realizando vacuum
2024-09-25 19:05:33,449 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [8]:
extrator_bronze.parar_sessao()

2024-09-25 19:05:33,904 - INFO - Sessão Spark finalizada.
